In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

# 1차시 

이미지 크기 변경 : 128 

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

In [0]:
import tensorflow as tf
from keras.applications import Xception
from keras.utils import multi_gpu_model
from keras import backend as K
import numpy as np

In [0]:
import pandas as pd
import numpy as np
import keras
import glob
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
#from mlxtend.preprocessing import minmax_scaling
from sklearn.metrics import roc_curve, auc

from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, Input, BatchNormalization, Multiply, Activation
from keras.optimizers import RMSprop, SGD
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras import backend as K
import tensorflow as tf

import os


In [0]:
# 변수 
model_name = "test_1"
model_save = f'{model_name}.hdf5'

PATH="/content/drive/My Drive/Colab Notebooks/data"
img_size = 128 
batch_size = 64

calsses_num = 4

In [0]:

# train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.33)

train_datagen = ImageDataGenerator(featurewise_center=False,
                 samplewise_center=False,
                 featurewise_std_normalization=False,
                 samplewise_std_normalization=False,
                 zca_whitening=False,
                 rotation_range=5,
                 width_shift_range=0.05,
                 height_shift_range=0.05,
                 shear_range=0.2,
                 zoom_range=0.2,
                 channel_shift_range=0.,
                 fill_mode='nearest',
                 cval=0.,
                 horizontal_flip=True,
                 vertical_flip=False,
                 rescale=1/255,
                validation_split=0.33
                                  )

In [0]:
train_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13,
        target_size=(img_size,img_size),
        batch_size=batch_size,
        class_mode = 'categorical',
        subset="training")


val_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13, 
        target_size=(img_size,img_size), 
        batch_size=batch_size,
        class_mode = 'categorical',
        subset="validation")


In [0]:
with tf.device("/device:GPU:0"):
    model = Sequential()

    model.add(Conv2D(filters = 256, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu', input_shape = (img_size,img_size,3), kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 128, kernel_size = (5,5), strides = 2, padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters = 64, kernel_size = (2,2),padding = 'Same', activation ='relu',kernel_initializer='he_normal'))
    model.add(GlobalAveragePooling2D())
    model.add(BatchNormalization())
    model.add(Dense(128, activation = "relu",kernel_initializer='he_normal'))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation = "relu",kernel_initializer='he_normal'))
    model.add(Dropout(0.2))
    model.add(Dense(calsses_num, activation = "softmax",kernel_initializer='he_normal',kernel_regularizer=l2()))
    model.summary()

    #callbacks
    checkpointer = ModelCheckpoint(filepath=model_save, verbose=1, save_best_only=True, save_weights_only=True)
    earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=20, mode='auto')
    reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, mode='auto')
    # stpes_per_epoch = 2250/64


    model.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics=["accuracy"])

In [0]:
    # opt = Adam(lr=0.0002, beta_1=0.5)
    # model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["accuracy"])
    # model.compile(optimizer = 'rmsprop' , loss = "categorical_crossentropy", metrics=["accuracy"])

In [0]:
with tf.device("/device:GPU:0"):
    history = model.fit_generator(train_generator,steps_per_epoch=100,
                            validation_data=val_generator,validation_steps=100, 
                            epochs=100 ,workers=0)

In [0]:
    #testing phase
